<a href="https://colab.research.google.com/github/sasidn/BigData/blob/main/02_dataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stemmer
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement stemmer (from versions: none)
ERROR: No matching distribution found for stemmer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import html
import re
#library that contains punctuation
import string
string.punctuation

# nltk
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk.stem as stemmer
#wordcloud
#from wordcloud import WordCloud, STOPWORDS
#importing nlp library
import nltk
nltk.download('stopwords')
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

# importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer

# defining the object for stemming
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet


def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    #for word in words:
        #word = preprocess_word(word)
        #if is_valid_word(word):
            #if use_stemmer:
                #word = str(porter_stemmer.stem(word))
            #processed_tweet.append(word)

    #return ' '.join(processed_tweet)
    return tweet

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#Tokenization:
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text



In [8]:
# read in the data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
cd /content/drive/My Drive/BIgDataProject/

/content/drive/My Drive/BIgDataProject


In [12]:
df = pd.read_csv('tweets.csv')

In [13]:
df.head(2)

,Unnamed: 0,Datetime,Text,Username,lang
0,0,2021-05-30 23:59:52+00:00,@JoeBarri @DawnRubino I am vaccinated. I have ...,jeleblanc15,en
1,1,2021-05-30 23:59:46+00:00,Pastor Rick Wiles Apparently Has COVID After R...,ADAMXMAN,en


In [19]:
#storing the puntuation free text
df['clean_msg']= df['Text'].apply(lambda x:remove_punctuation(x))
df['preprocess_msg']= df['Text'].apply(lambda x:preprocess_word(x))

df['preprocess_tweet']= df['Text'].apply(lambda x:preprocess_tweet(x))
#Lowering the text:
df['msg_lower']= df['clean_msg'].apply(lambda x: x.lower())
df['msg_tokenied']= df['msg_lower'].apply(lambda x: tokenization(x))
df['no_stopwords']= df['msg_tokenied'].apply(lambda x:remove_stopwords(x))
df['msg_stemmed']=df['no_stopwords'].apply(lambda x: stemming(x))
df['msg_lemmatized']=df['no_stopwords'].apply(lambda x:lemmatizer(x))
#df.drop(columns=['is_valid_word'],inplace=True)
df.drop(columns=['preprocess_tweet'],inplace=True)
#df.drop(columns=['handle_emojis'],inplace=True)
df.drop(columns=['msg_lower'],inplace=True)
df.drop(columns=['msg_tokenied'],inplace=True)
df.drop(columns=['no_stopwords'],inplace=True)
df.drop(columns=['msg_stemmed'],inplace=True)
df.drop(columns=['msg_lemmatized'],inplace=True)

In [17]:
df.head(2)

,Unnamed: 0,Datetime,Text,Username,lang,clean_msg,preprocess_msg
0,0,2021-05-30 23:59:52+00:00,@JoeBarri @DawnRubino I am vaccinated. I have ...,jeleblanc15,en,JoeBarri DawnRubino I am vaccinated I have had...,@JoeBarri @DawnRubino I am vaccinated. I have ...
1,1,2021-05-30 23:59:46+00:00,Pastor Rick Wiles Apparently Has COVID After R...,ADAMXMAN,en,Pastor Rick Wiles Apparently Has COVID After R...,Pastor Rick Wiles Apparently Has COVID After R...
